In [1]:
# Importing Libraries : 
import pandas as pd
import random
import spacy
from spacy.training import Example
from spacy.pipeline.ner import DEFAULT_NER_MODEL
import re
from time import sleep
import re
from spacy import logger
from spacy import displacy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report


In [2]:
# Loading the dataset : 
training_file = 'training.txt'
testing_file  = 'test.txt'

with open("training.txt", "r") as train_file:
    training_data = train_file.read()

with open("test.txt", "r") as test_file:
    testing_data = test_file.read()

# Training set : bifuricating the text based on the title and annotations
ctr = 0
train = []
for line in training_data.split("\n\n"):
  train.append(line)

# Test Set:
ctr = 0
test = []
for line in testing_data.split("\n\n"):
  test.append(line)

In [3]:
# Extracting the Titles and Makig the annotations : 
try:

  def article_extractor(text):
    article = re.findall(r'a\|(.*)\n' , text)
    return article[0]

  def title_extractor(text):
    title = re.findall(r't\|(.*)\n' , text)
    return title[0]

except Exception as e:
  pass

train_article  = []
try:
  for x in train:
    train_article.append(title_extractor(x)+' '+article_extractor(x))
except Exception as e:
  pass

test_article  = []
try:
  for x in test:
    test_article.append(title_extractor(x)+' '+article_extractor(x))
except Exception as e:
  pass

In [4]:
# Selecting the labels from the dataset :

try:
  def get_labels(text):
    l  = re.findall(r'\t(.*)' , text)
    l = [x.split('\t') for x in l]
    labels  = []
    for i in l:
      try:
        labels.append((int(i[0]) , int(i[1]) , i[3]))
      except Exception as e:
        pass
    return labels


  def get_labels_and_entity(text):
    l = re.findall(r'\t(.*)' , text)
    l = [x.split('\t') for x in l]
    labels  = []
    for i in l:
      try:
        labels.append((int(i[0]) , int(i[1]) , i[3], i[2]))
      except Exception as e:
        pass
    return labels
except Exception as e:
  pass

train_labels  = [get_labels(x) for x in train]
test_labels = [get_labels(x) for x in test]

train_labels = [{"entities": entry} for entry in train_labels]
test_labels = [{"entities": entry} for entry in test_labels] 

# The entities text is added in the dataset as training spacy model needs to have a specifi format :

In [5]:
# Making the data frame : 

train_df  = pd.DataFrame(train_article, columns=['article'])
test_df  = pd.DataFrame(test_article, columns=['article'])

min_length = min(len(test_df), len(test_labels), len(train_df), len(train_labels))

test_df = test_df.iloc[:min_length].copy()
train_df = train_df.iloc[:min_length].copy()

test_df['labels'] = test_labels[:min_length]
train_df['labels'] = train_labels[:min_length]


In [6]:
train_data  = []
test_data = []

for _, row in train_df.iterrows():
    text = row['article']
    entities = row['labels']['entities'] 
    train_data.append((text, {"entities": entities}))

# print(train_data[0])

for _, row in test_df.iterrows():
    text = row['article']
    entities = row['labels']['entities']
    
    test_data.append((text, {"entities": entities}))


In [7]:
test_data[0]

('Famotidine-associated delirium. A series of six cases. Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.',
 {'entities': [(0, 10, 'Chemical'),
   (22, 30, 'Disease'),
   (55, 65, 'Chemical'),
   (156, 162, 'Disease'),
   (324, 332, 'Disease'),
   (395, 405, 'Chemical'),
   (442, 452, 'Chemical'),
   (464, 472, 'Disease'),
   (537, 547, 'Chemical'),
   (573, 583, 'Che

In [8]:
train_data[0]

('Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release

In [9]:
### Both the train and test dataset are done and now we can train the model : 
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

nlp.initialize()

def train_ner(nlp, train_data, iterations=10):
    optimizer = nlp.create_optimizer()
    for epoch in range(iterations):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in train_data:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], drop=0.4, losses=losses)
        print(f"Epoch {epoch+1}, Loss: {losses}")

In [10]:

# Call the train ner function : for training the model :
train_ner(nlp, train_data)

c:\Users\vinay\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Epithelial sodium channel (ENaC) subunit mRNA and ..." with entities "[(11, 17, 'Chemical'), (82, 107, 'Chemical'), (116...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\vinay\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Conformationally restricted analogs of BD1008 and ..." with entities "[(39, 45, 'Chemical'), (63, 83, 'Chemical'), (124,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\vinay\AppData\Local\Programs\Python\Python310

Epoch 1, Loss: {'ner': np.float32(14481.574)}
Epoch 2, Loss: {'ner': np.float32(8381.821)}
Epoch 3, Loss: {'ner': np.float32(6577.9463)}
Epoch 4, Loss: {'ner': np.float32(5298.951)}
Epoch 5, Loss: {'ner': np.float32(4530.48)}
Epoch 6, Loss: {'ner': np.float32(3951.384)}
Epoch 7, Loss: {'ner': np.float32(3555.3608)}
Epoch 8, Loss: {'ner': np.float32(3184.35)}
Epoch 9, Loss: {'ner': np.float32(3097.5032)}
Epoch 10, Loss: {'ner': np.float32(2675.3225)}


In [11]:
# Make function for the uncertain predictions and feed some unlabeled data in the model : 

def get_uncertain_predictions(nlp, texts, threshold=0.5):
    uncertain_samples = []
    for text in texts:
        doc = nlp(text)
        for ent in doc.ents:
            # Simulate uncertainty based on entity length (longer names -> more confidence)
            confidence = len(ent.text) / len(text)
            if confidence < threshold:
                uncertain_samples.append((text, ent.text, ent.label_, confidence))
    return uncertain_samples

UNLABELED_DATA = [
    "Ibuprofen is a pain reliever.",
    "Clonidine is used for blood pressure control.",
    "Patients with diabetes should monitor sugar levels.",
]

uncertain_samples = get_uncertain_predictions(nlp, UNLABELED_DATA)


In [12]:
test_data[0][0]

'Famotidine-associated delirium. A series of six cases. Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost. Although all of the currently available H2-receptor antagonists have shown the propensity to cause delirium, only two previously reported cases have been associated with famotidine. The authors report on six cases of famotidine-associated delirium in hospitalized patients who cleared completely upon removal of famotidine. The pharmacokinetics of famotidine are reviewed, with no change in its metabolism in the elderly population seen. The implications of using famotidine in elderly persons are discussed.'

In [13]:
# Model Testing : setting the data using displacy :: 
colors  = {
    "Disease" : "#ff698f",
    "Chemical" : "#1472ff"
}

options  = {"ents" : ["Disease" , "Chemical"] , "colors" : colors}

In [14]:
def save_model(nlp, model_path):
    nlp.to_disk(model_path)
    print(f"Model saved to {model_path}")

save_model(nlp=nlp , model_path="saved_model")

Model saved to saved_model


In [15]:
# Check the model for some random text :: from testing data ::
# Lets check the test for the first 10 entries and see the output : 

nlp = spacy.load('saved_model')

for i in range(5):
    text = test_data[i][0]
    doc  = nlp(text=text)
    spacy.displacy.render(doc ,style  = "ent" , jupyter = True  , options = options)


In [18]:

model_path =   'saved_model'
check_model  = spacy.load(model_path)

sample_text = input("Enter Medical Based Text  :")

doc = check_model(sample_text)
spacy.displacy.render(doc ,style  = "ent" , jupyter = True , options = options )